In [357]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [358]:
import sys
import os
import torch
from functools import partial
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import torch.nn as nn
from embedding import CustomEmbedding
from transformers import EncoderDecoderTransformer
from utils import padding_collate_fn

from generate_data import RandomIntegerDataset

In [359]:
n_real_tokens = 10
PAD_TOKEN_IDX = n_real_tokens
SOS_TOKEN_IDX = n_real_tokens + 1
EOS_TOKEN_IDX = n_real_tokens + 2
vocab_size = n_real_tokens + 3
D_MODEL = 32

embeddings = CustomEmbedding(vocab_size, d_model = D_MODEL) # 3 = PAD, SOS, EOS

indices = torch.tensor([1,9])

# print(embeddings.embeddings.weight)
print(embeddings.embeddings(indices))

tensor([[-0.2462, -0.3919, -0.0947,  0.7233,  0.3639,  0.5125, -0.9399,  0.5011,
         -1.4048,  1.6272,  0.3549,  0.9267,  0.2432, -1.3953,  0.7241,  0.2306,
          0.1970,  0.0726,  0.9761,  1.2269,  1.9732,  0.4609, -0.1448,  0.8718,
         -2.2125,  0.1273,  1.0215,  0.0922, -1.4178, -0.3489,  1.1586, -2.0161],
        [-0.8305,  1.4949, -0.2491, -0.4186,  1.8919, -0.0442, -1.0363,  0.0434,
          0.1472,  1.2258,  2.5006, -1.1245,  1.6088,  0.9654,  3.2616,  1.0740,
         -0.8235,  0.4351,  0.1763, -0.4741, -0.3765,  0.6691, -0.1097, -0.1007,
         -0.0537, -0.6360, -0.1322,  0.3438,  0.9376,  2.8977,  1.1563, -1.0842]],
       grad_fn=<EmbeddingBackward0>)


In [360]:
MAX_CONTEXT_WINDOW = 50

BATCH_SIZE = 32
MIN_SEQ_LEN = 2
MAX_SEQ_LEN = min(10, MAX_CONTEXT_WINDOW)

NUM_TRAINING_SEQUENCES = 10000
NUM_VALIDATION_SEQUENCES = 1000

VOCAB = [i for i in range(n_real_tokens)] # non-sos,eos,pad

train_rand_ds = RandomIntegerDataset(MIN_SEQ_LEN, MAX_SEQ_LEN, NUM_TRAINING_SEQUENCES, VOCAB)
train_dataloader = DataLoader(train_rand_ds, batch_size = BATCH_SIZE, shuffle = True, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

val_rand_ds = RandomIntegerDataset(MIN_SEQ_LEN, MAX_SEQ_LEN, NUM_VALIDATION_SEQUENCES, VOCAB)
val_dataloader = DataLoader(val_rand_ds, batch_size = BATCH_SIZE, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

In [361]:
input, label = next(iter(train_dataloader))
print(input[0])
print(input[1])
print(label)

tensor([[ 0,  8, 10, 10, 10, 10, 10, 10, 10, 10],
        [ 5,  5,  9,  7,  5,  6,  9,  1,  5,  2],
        [ 2,  6,  4,  5,  9,  2, 10, 10, 10, 10],
        [ 1,  4,  8,  6,  0,  7,  0,  6, 10, 10],
        [ 1,  1,  7, 10, 10, 10, 10, 10, 10, 10],
        [ 7,  7,  2, 10, 10, 10, 10, 10, 10, 10],
        [ 6,  0,  5,  3,  9,  2, 10, 10, 10, 10],
        [ 2,  9,  8,  9,  4,  3,  1,  6,  5,  2],
        [ 8,  4,  0,  7,  0,  8, 10, 10, 10, 10],
        [ 5,  5,  2, 10, 10, 10, 10, 10, 10, 10],
        [ 5,  5,  1,  8,  5,  6,  9,  3, 10, 10],
        [ 4,  5,  1,  8,  5, 10, 10, 10, 10, 10],
        [ 0,  0,  7,  5, 10, 10, 10, 10, 10, 10],
        [ 1,  1,  8, 10, 10, 10, 10, 10, 10, 10],
        [ 0,  2,  3,  7,  9,  4,  6,  6, 10, 10],
        [ 9,  0,  8,  7, 10, 10, 10, 10, 10, 10],
        [ 1,  4,  4,  3,  6, 10, 10, 10, 10, 10],
        [ 6,  6,  0,  9,  6,  0,  7,  0,  0,  1],
        [ 0,  9,  4,  7,  7,  2,  2,  7,  5,  7],
        [ 0,  7,  5,  2,  8,  1,  2,  9,  0,  2],


In [362]:
loss_fn = nn.CrossEntropyLoss(ignore_index = PAD_TOKEN_IDX, reduction = 'sum')

model = EncoderDecoderTransformer(
                    embeddings = embeddings, 
                    vocab_size = vocab_size, 
                    d_model = D_MODEL, 
                    num_attention_heads = 4, 
                    num_encoder_layers = 1, 
                    num_decoder_layers = 1, 
                    dim_feedforward = 32, 
                    dropout = 0.0,
                    max_context_window = MAX_CONTEXT_WINDOW)

optim = torch.optim.SGD(params = model.parameters(), lr = 1e-3, momentum = 0.9, weight_decay = 1e-4)

In [363]:
def run_train_epoch(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, optimizer: torch.optim.Optimizer, calculate_sequence_accuracy: bool = False, calculate_token_accuracy: bool = False):
    """
    """
    model.train()

    num_sequences = len(dataloader.dataset)
    num_tokens = 0

    epoch_loss = 0.0
    total_correct_sequences = 0
    total_correct_tokens = 0

    for (source, target), label in tqdm(dataloader):

        # FORWARD
        pred_logits = model(source, target)

        # pred_logits.shape: [batch_size, seq_len, vocab_size]
        # label.shape: [batch_size, seq_len]

        # CrossEntropyLoss (loss_fn) only takes 2D predictions (n_batch * seq_len, vocab_size) and 1D labels (n_batch * seq_len)
        batch_loss = loss_fn(pred_logits.view(-1, pred_logits.size(-1)), label.view(-1))

        # LOG
        with torch.no_grad():
            epoch_loss += batch_loss.item()

            predictions = torch.argmax(pred_logits, dim = -1) # predictions.shape: [batch_size, seq_len]
            match_matrix = torch.eq(predictions, label)

            if calculate_sequence_accuracy:
                num_correct_sequences = torch.all(match_matrix, dim = 1).sum()
                total_correct_sequences += num_correct_sequences.item()

            if calculate_token_accuracy:
                num_correct_tokens = match_matrix.sum()      
                total_correct_tokens += num_correct_tokens.item()

                num_tokens += torch.numel(label)

        # BACKWARD
        batch_loss.backward()

        # OPTIMIZE
        optimizer.step()
        optimizer.zero_grad()

    average_epoch_loss = epoch_loss / num_sequences
    average_epoch_sequence_accuracy = total_correct_sequences / num_sequences if calculate_sequence_accuracy else None
    average_epoch_token_accuracy = total_correct_tokens / num_tokens if calculate_token_accuracy else None

    return average_epoch_loss, average_epoch_sequence_accuracy, average_epoch_token_accuracy

def run_gold_validation_loop(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, calculate_sequence_accuracy: bool = False, calculate_token_accuracy: bool = False):
    """
    """
    model.eval()

    num_sequences = len(dataloader.dataset)
    num_tokens = 0

    epoch_loss = 0.0
    total_correct_sequences = 0
    total_correct_tokens = 0

    with torch.no_grad():
        
        for (source, target), label in tqdm(dataloader):
            
            # FORWARD
            pred_logits = model(source, target)
            batch_loss = loss_fn(pred_logits.view(-1, pred_logits.size(-1)), label.view(-1))

            # LOG
            epoch_loss += batch_loss.item()

            predictions = torch.argmax(pred_logits, dim = -1) # predictions.shape: [batch_size, seq_len]
            match_matrix = torch.eq(predictions, label)

            if calculate_sequence_accuracy:
                num_correct_sequences = torch.all(match_matrix, dim = 1).sum()
                total_correct_sequences += num_correct_sequences.item()

            if calculate_token_accuracy:
                num_correct_tokens = match_matrix.sum()      
                total_correct_tokens += num_correct_tokens.item()

                num_tokens += torch.numel(label)

    average_epoch_loss = epoch_loss / num_sequences
    average_epoch_sequence_accuracy = total_correct_sequences / num_sequences if calculate_sequence_accuracy else None
    average_epoch_token_accuracy = total_correct_tokens / num_tokens if calculate_token_accuracy else None

    return average_epoch_loss, average_epoch_sequence_accuracy, average_epoch_token_accuracy

def run_autoregressive_validation_loop(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module):
    """
    """
    model.eval()

    with torch.no_grad():
        ...

In [364]:
EPOCHS = 10

training_losses = list()
training_sequence_accuracies = list()
training_token_accuracies = list()

gold_validation_losses = list()
gold_validation_sequence_accuracies = list()
gold_validation_token_accuracies = list()

for i in range(EPOCHS):
    # print(f'Running epoch {i+1}...')

    training_loss, training_sequence_accuracy, training_token_accuracy = run_train_epoch(train_dataloader, model, loss_fn, optim, calculate_sequence_accuracy = True, calculate_token_accuracy = True)

    training_losses.append(training_loss)
    training_sequence_accuracies.append(training_sequence_accuracy)
    training_token_accuracies.append(training_token_accuracy)

    gold_val_loss, gold_val_sequence_accuracy, gold_val_token_accuracy = run_gold_validation_loop(val_dataloader, model, loss_fn, calculate_sequence_accuracy = True, calculate_token_accuracy = True)
    gold_validation_losses.append(gold_val_loss)
    gold_validation_sequence_accuracies.append(gold_val_sequence_accuracy)
    gold_validation_token_accuracies.append(gold_val_token_accuracy)

print(training_losses)
print(training_sequence_accuracies)
print(training_token_accuracies)

print()

print(gold_validation_losses)
print(gold_validation_sequence_accuracies)
print(gold_validation_token_accuracies)

100%|██████████| 32/32 [00:00<00:00, 309.64it/s]

[3.911926256561279, 2.4808859935760497, 2.0879380752563477, 1.9818954223632812, 2.0528027486801146, 1.9057706783294677, 1.6326018392562867, 1.781339628648758, 1.5982161798477172, 1.4132523089408875]
[0.0036, 0.0102, 0.0136, 0.0203, 0.018, 0.0183, 0.0296, 0.0264, 0.0308, 0.0366]
[0.4988953403447268, 0.550673455978975, 0.5625819373634378, 0.5694768714431636, 0.5652490533061463, 0.5713387520046654, 0.5824704681347528, 0.5750182083029861, 0.5825550853640741, 0.5887334449134042]

[3.518376655578613, 1.9308992319107057, 1.8528783473968506, 2.1387152557373046, 4.099720119476318, 2.8950458717346192, 0.9730916028022766, 0.908331072807312, 0.9061744184494018, 1.5064511165618897]
[0.0, 0.024, 0.015, 0.007, 0.004, 0.024, 0.075, 0.071, 0.062, 0.026]
[0.5237964989059081, 0.5718453683442742, 0.5787746170678337, 0.5697483588621444, 0.5295404814004376, 0.550419401896426, 0.6153355215171408, 0.6133296863603209, 0.6136943836615609, 0.599653537563822]
